В этом блокноте мы занимаемся скачиванием продуктов с сайта Ленты

Для начала нам нужно импортировать нужные библиотеки и задать константы. Это параметры запросов к сайту.

In [1]:
import requests
import json
import time
import random
import pandas as pd
from datetime import date

# define constants for requests

CATEGORIES_URL = "https://lenta.com/api-gateway/v1/catalog/categories?timestamp="
CHECK_DATE = date.today() # current date for reference

HEADERS = {
    'DeviceID': '42b24bb8-650a-f3a7-fa8d-593e40b478d4',
    'Experiments': 'exp_recommendation_cms.true, exp_apigw_purchase.test, exp_lentapay.test, exp_omni_price.test, exp_profile_bell.test, exp_newui_cancel_order.test, exp_newui_history_active_action.test_stars, exp_comment_picker_and_courier.test, exp_general_editing_page.test, exp_cl_omni_support.test, exp_cl_omni_authorization.test, exp_onboarding_sbp.default, exp_fullscreen.test, exp_profile_login.false, exp_new_notifications_show_unauthorized.test, exp_assembly_cost_location.cart, exp_search_bottom.default, exp_onboarding_editing_order.test, exp_cart_new_carousel.default, exp_newui_cart_cancel_editing.test, exp_newui_cart_button.test, exp_new_promov3., exp_sbp_enabled.test, exp_new_my_goods.test, exp_ui_catalog.test, exp_search_out_of_stock.default, exp_profile_settings_email.default, exp_cl_omni_refusalprintreceipts.test, exp_cl_omni_refusalprintcoupons.test, exp_accrual_history.test, exp_personal_recommendations.control, exp_newui_chips.test, exp_loyalty_categories.test, exp_growthbooks_aa.OFF, exp_test_ch_web.def, exp_search_suggestions_popular_sku.default, exp_cancel_subscription.test_2, exp_manage_subscription.control, exp_cl_new_csi.default, exp_cl_new_csat.default, exp_delivery_price_info.default, exp_personal_promo_navigation.test, exp_web_feature_test.true, exp_interval_jump.default, exp_cardOne_promo_type.test, exp_qr_cnc.test, exp_popup_about_order.test, exp_apigw_recommendations.test, exp_where_place_cnc.control, exp_editing_cnc_onboarding.default, exp_editing_cnc.default, exp_selection_carousel.test, exp_pickup_in_delivery.false, exp_feature_kpp_test.false, exp_welcome_onboarding.default, exp_cl_new_splash.default, exp_web_referral_program_type.default, exp_where_place_new.default, exp_start_page.default, exp_promocode_bd_coupon.default, exp_personal_promo_swipe_animation.default, exp_default_payment_type.default, exp_main_page_carousel_vs_banner.default, exp_start_page_onboarding.default, exp_newui_cart_check_edit.default, exp_search_new_logic.default, exp_search_ds_pers_similar.default, exp_growthbooks_aa_id_based_feature.control, exp_referral_program_type.default, exp_my_choice_search.default, exp_items_by_rating.default, exp_can_accept_early.default, exp_test_gb_value.false, exp_online_subscription.default, exp_new_nps_keyboard.test, exp_main_page_carousel_vs_banner_shop.default, exp_bathcing.default, exp_web_qr_cnc.default, exp_hide_cash_payment_for_cnc_wo_adult_items.default, exp_web_promocode_bd_coupon.default, exp_prices_per_quantum.default, exp_test.default123, exp_web_partner_coupons_separately.default, exp_web_chips_online.default',
    'SessionToken': 'E05BD3D1115FF4A5D5F0BC02D141EB5A',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36',
    'X-Delivery-Mode': 'pickup',
    'X-Platform': 'omniweb',
    'X-Retail-Brand': 'lo',
}

In [2]:
HEADERS['Referer'] = f'https://lenta.com/product/apelsiny-dlya-soka-fasovannye-vesovye-023669/'
response = requests.get(f'https://lenta.com/api-gateway/v1/catalog/items/023669?timestamp={int(time.time() * 1000)}', headers=HEADERS) # get the data
raw_categories = json.loads(response.text) # convert into a dictionary
raw_categories

{'attributes': [{'alias': 'brand',
   'id': 238,
   'name': 'Бренд',
   'slug': 'brand',
   'value': 'Прочие Товары',
   'valueId': 10720665,
   'valueSlug': 'prochie-tovary'},
  {'alias': '',
   'id': 84,
   'name': 'Полное название товара',
   'slug': 'polnoe-nazvanie-tovara',
   'value': 'Апельсины для сока, фасованные, весовые',
   'valueId': 6813914,
   'valueSlug': 'apelsiny-dlya-soka-fasovannye-vesovye'},
  {'alias': '',
   'id': 180,
   'name': 'Стандарт качества',
   'slug': 'standart-kachestva',
   'value': 'ТР ТС',
   'valueId': 6755911,
   'valueSlug': 'tr-ts'},
  {'alias': '',
   'id': 221,
   'name': 'Сорт',
   'slug': 'sort',
   'value': 'Высший',
   'valueId': 6751545,
   'valueSlug': 'vysshijj'},
  {'alias': 'vendorId',
   'id': 236,
   'name': 'Артикул',
   'slug': 'artikul',
   'value': '023669',
   'valueId': 11029223,
   'valueSlug': '023669'},
  {'alias': 'description',
   'id': 242,
   'name': 'Описание',
   'slug': 'opisanie',
   'value': 'Апельсины для сока – я

Так как скачивать продукты мы будем по категориям, нужно получить список категорий товаров с сайта. За это будет отвечать следующая функция, которая запросит у сайта список категорий и подкатегорий, выберет только id и названия категорий и сохранит данные в словарь.

In [157]:
def fetch_categories():
    
    response = requests.get(f'{CATEGORIES_URL}{int(time.time() * 1000)}', cookies=COOKIES, headers=HEADERS) # get the data
    raw_categories = json.loads(response.text) # convert into a dictionary
    
    cleaned_categories = []
    
    # go through the raw data and select only the necessary fields
    for category in raw_categories["categories"]:    # select the broad category, save its id and name
        cleaned_categories.append({
            "id": category["id"],
            "name": category["name"],
            "parent_id": category["parentId"] if category["parentId"] != 0 else None,
            "parent_name": category["parentName"] if category["parentName"] != '' else None,
            "slug": category["slug"], # to create a referer for further items fetching
            "level": category["level"] # to avoid double scanning subcategories
        })

    return cleaned_categories

Загружаем категории в отдельном блоке, чтобы не повторять эту операцию.

In [158]:
categories = fetch_categories()
categories_df = pd.DataFrame(categories)
categories_df.to_csv(f'categories-{CHECK_DATE}-lenta.csv', index=False)
print(categories)

[{'id': 20638, 'name': 'Постно и полезно', 'parent_id': None, 'parent_name': None, 'slug': 'postno-i-polezno', 'level': 1}, {'id': 20459, 'name': 'Дарите цветы', 'parent_id': None, 'parent_name': None, 'slug': 'darite-cvety', 'level': 1}, {'id': 17036, 'name': 'Алкоголь', 'parent_id': None, 'parent_name': None, 'slug': 'alkogol', 'level': 1}, {'id': 1893, 'name': 'Особенно выгодно', 'parent_id': None, 'parent_name': None, 'slug': 'osobenno-vygodno', 'level': 1}, {'id': 19305, 'name': 'Новинки', 'parent_id': None, 'parent_name': None, 'slug': 'novinki', 'level': 1}, {'id': 18427, 'name': 'Товары до 99 рублей', 'parent_id': None, 'parent_name': None, 'slug': 'tovary-do-99-rublejj', 'level': 1}, {'id': 42, 'name': 'Готовая еда', 'parent_id': None, 'parent_name': None, 'slug': 'gotovaya-eda', 'level': 1}, {'id': 18359, 'name': 'Только в Ленте', 'parent_id': None, 'parent_name': None, 'slug': 'tolko-v-lente', 'level': 1}, {'id': 144, 'name': 'Овощи и фрукты', 'parent_id': None, 'parent_name

In [167]:
categories_2 = [cat for cat in categories if cat['level'] == 2] # choose only level 2 categories
categories_2_cut = categories_2[60:]
print(categories_2_cut)
total_categories_count = len(categories_2_cut)
total_categories_count

[{'id': 1074, 'name': 'Средства для стирки', 'parent_id': 1070, 'parent_name': 'Бытовая химия', 'slug': 'sredstva-dlya-stirki', 'level': 2}, {'id': 1169, 'name': 'Замороженная рыба', 'parent_id': 183, 'parent_name': 'Рыба, морепродукты', 'slug': 'zamorozhennaya-ryba', 'level': 2}, {'id': 1446, 'name': 'Чайники и термопоты', 'parent_id': 1439, 'parent_name': 'Бытовая техника', 'slug': 'chajjniki-i-termopoty', 'level': 2}, {'id': 1495, 'name': 'Посуда для сервировки', 'parent_id': 1427, 'parent_name': 'Кухня', 'slug': 'posuda-dlya-servirovki', 'level': 2}, {'id': 1756, 'name': 'Удобрения и грунты', 'parent_id': 1731, 'parent_name': 'Дача и сад', 'slug': 'udobreniya-i-grunty', 'level': 2}, {'id': 2977, 'name': 'Мягкий, плавленый, творожный', 'parent_id': 2, 'parent_name': 'Сыры', 'slug': 'myagkijj-plavlenyjj-tvorozhnyjj', 'level': 2}, {'id': 3019, 'name': 'Велоспорт', 'parent_id': 1419, 'parent_name': 'Спорт и отдых', 'slug': 'velosport', 'level': 2}, {'id': 3023, 'name': 'Туризм и рыбалк

374

In [168]:
ITEMS_HEADERS = {
    'DeviceID': '7fcf1400-3d1b-303b-4421-e8f7bb483e3e',
    'Experiments': 'exp_recommendation_cms.true, exp_apigw_purchase.test, exp_lentapay.test, exp_omni_price.test, exp_profile_bell.test, exp_newui_cancel_order.test, exp_newui_history_active_action.test_stars, exp_comment_picker_and_courier.test, exp_general_editing_page.test, exp_cl_omni_support.test, exp_cl_omni_authorization.test, exp_onboarding_sbp.default, exp_fullscreen.test, exp_profile_login.false, exp_new_notifications_show_unauthorized.test, exp_assembly_cost_location.cart, exp_search_bottom.default, exp_onboarding_editing_order.test, exp_cart_new_carousel.default, exp_newui_cart_cancel_editing.test, exp_newui_cart_button.test, exp_new_promov3., exp_sbp_enabled.test, exp_new_my_goods.test, exp_ui_catalog.test, exp_search_out_of_stock.default, exp_profile_settings_email.default, exp_cl_omni_refusalprintreceipts.test, exp_cl_omni_refusalprintcoupons.test, exp_accrual_history.test, exp_personal_recommendations.test_A, exp_newui_chips.test, exp_loyalty_categories.test, exp_growthbooks_aa.OFF, exp_test_ch_web.def, exp_search_suggestions_popular_sku.default, exp_cancel_subscription.test_2, exp_manage_subscription.control, exp_cl_new_csi.default, exp_cl_new_csat.default, exp_delivery_price_info.default, exp_personal_promo_navigation.test, exp_web_feature_test.true, exp_interval_jump.default, exp_cardOne_promo_type.test, exp_qr_cnc.contol, exp_popup_about_order.contol, exp_apigw_recommendations.test, exp_where_place_cnc.test, exp_editing_cnc_onboarding.default, exp_editing_cnc.default, exp_selection_carousel.test, exp_pickup_in_delivery.false, exp_feature_kpp_test.false, exp_welcome_onboarding.default, exp_cl_new_splash.default, exp_web_referral_program_type.default, exp_where_place_new.default, exp_start_page.default, exp_promocode_bd_coupon.default, exp_personal_promo_swipe_animation.default, exp_default_payment_type.default, exp_main_page_carousel_vs_banner.default, exp_start_page_onboarding.default, exp_newui_cart_check_edit.default, exp_search_new_logic.default, exp_search_ds_pers_similar.default, exp_growthbooks_aa_id_based_feature.control, exp_referral_program_type.default, exp_my_choice_search.default, exp_items_by_rating.default, exp_can_accept_early.default, exp_test_gb_value.false, exp_online_subscription.default, exp_new_nps_keyboard.test, exp_main_page_carousel_vs_banner_shop.default, exp_bathcing.default, exp_web_qr_cnc.default, exp_hide_cash_payment_for_cnc_wo_adult_items.default, exp_web_promocode_bd_coupon.default, exp_prices_per_quantum.default, exp_test.default123, exp_web_partner_coupons_separately.default, exp_web_chips_online.default',
    'SessionToken': '8F8747EF502EC8C788B958A7F851ADD0',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36 Edg/133.0.0.0',
    'Referer': '',
    'X-Delivery-Mode': 'pickup',
    'X-Platform': 'omniweb',
    'X-Retail-Brand': 'lo',
}

In [169]:
# fetch the raw product data (the site uses an increasing offset to fetch all the products, we mimic this behavior)

def fetch_products(category):
    
    total_products = 1 # just a random non-zero value to start with
    offset = 0
    products = []

    ITEMS_HEADERS['Referer'] = f'https://lenta.com/catalog/{category["slug"]}-{category["id"]}/'
    url = 'https://lenta.com/api-gateway/v1/catalog/items'
        
    while offset < total_products: # loop until offset doesn't
        time.sleep(random.uniform(1, 5)) # random time delay to avoid being blocked
        
        json_data = {
            'categoryId': category['id'],
            'limit': 40,
            'offset': offset,
            'sort': {
                'type': 'popular',
                'order': 'desc',
            },
            'filters': {
                'range': [],
                'checkbox': [],
                'multicheckbox': [],
            },
        }
    
        response = requests.post(url, headers=HEADERS, json=json_data) # get the data
        response_data = json.loads(response.text) # convert into a dictionary
        products += response_data["items"] # extract only products and add them to the list
        total_products = response_data["total"] # update total number of products
        
        offset += 40 # update offset
        
        print(f'{len(products)}..', end='') # progress indication

    return products
    

In [170]:
def clean_product_data(category, raw_products):

    cleaned_products = []

    # go through the raw data and select only the necessary fields
    for product in raw_products:
        cleaned_products.append({
            "category_id": category["id"],
            "name": product["name"],
            "price": product["prices"]["priceRegular"] / 100,
            "pricing_unit": product['weight']["package"], # clarifies the unit for the price or the net weight
            "supermarket": 'Lenta'
        })

    return cleaned_products

In [172]:
fetched_categories_count = 0    # counter for fetching progress tracker

# Create the file first with headers
products_df = pd.DataFrame(columns=["category_id", "name", "price", "pricing_unit", "supermarket"])
products_df.to_csv(f'products-{CHECK_DATE}-lenta.csv', index=False, mode='w')

for category in categories_2_cut:
    raw_products = fetch_products(category) # fetch products
    new_products = clean_product_data(category, raw_products) # select only relevant data and add new products to the list

    products_df = pd.DataFrame(new_products)
    products_df.to_csv(f'products-{CHECK_DATE}-lenta.csv', index=False, mode='a', header=False)

    fetched_categories_count += 1
    print(f'№ {category["id"]} finished, {fetched_categories_count} out of {total_categories_count} categories fetched')
    time.sleep(random.uniform(1, 5))

print(f'Fetching complete, see the result in products-{CHECK_DATE}-lenta.csv')

40..80..120..160..200..240..280..320..360..400..421..№ 1074 finished, 1 out of 374 categories fetched
40..66..№ 1169 finished, 2 out of 374 categories fetched
34..№ 1446 finished, 3 out of 374 categories fetched
40..80..120..160..200..240..280..320..360..400..440..480..520..560..574..№ 1495 finished, 4 out of 374 categories fetched
40..75..№ 1756 finished, 5 out of 374 categories fetched
40..80..120..160..199..№ 2977 finished, 6 out of 374 categories fetched
34..№ 3019 finished, 7 out of 374 categories fetched
40..79..№ 3023 finished, 8 out of 374 categories fetched
40..78..№ 17983 finished, 9 out of 374 categories fetched
31..№ 17998 finished, 10 out of 374 categories fetched
40..80..120..160..200..225..№ 18031 finished, 11 out of 374 categories fetched
40..80..120..131..№ 18351 finished, 12 out of 374 categories fetched
40..80..120..160..182..№ 19208 finished, 13 out of 374 categories fetched
1..№ 19254 finished, 14 out of 374 categories fetched
40..80..117..№ 19328 finished, 15 out 

---

In [1]:
import pandas as pd
import re

In [2]:
categories = pd.read_csv('categories-2025-03-04-lenta.csv')
products_original = pd.read_csv('products-2025-03-04-lenta-complete.csv')
products = products_original.drop(['category_id'], axis=1)    # drop category_id column as it's actually unnecessary

In [5]:
categories

,id,name,parent_id,parent_name,slug,level
0,20638,Постно и полезно,NaN,NaN,postno-i-polezno,1
1,20459,Дарите цветы,NaN,NaN,darite-cvety,1
2,17036,Алкоголь,NaN,NaN,alkogol,1
3,1893,Особенно выгодно,NaN,NaN,osobenno-vygodno,1
4,19305,Новинки,NaN,NaN,novinki,1
...,...,...,...,...,...,...
1762,18646,Дезодоранты мужские,13929.0,Дезодоранты и антиперспиранты,dezodoranty-muzhskie,3
1763,18708,Выпечка и запекание,18703.0,Посуда,vypechka-i-zapekanie,3
1764,18710,"Чашки, кружки",18709.0,Посуда для напитков,chashki-kruzhki,3
1765,18711,Стаканы,18709.0,Посуда для напитков,stakany,3


In [6]:
products

,name,price,pricing_unit,supermarket
0,"Макароны PASTA ZARA Спагетти, 500г",121.05,500г,Lenta
1,"Макароны PASTA ZARA Рожки №27, 500г",121.05,500г,Lenta
2,Макароны ШЕБЕКИНСКИЕ Вермишель легкая группа А...,73.69,450г,Lenta
3,"Макароны ШЕБЕКИНСКИЕ Спирали группа А, высший ...",75.49,450г,Lenta
4,Лапша яичная РОЛЛТОН Классическая группа В выс...,78.94,400г,Lenta
...,...,...,...,...
49349,"Носки мужские INWIN р. 27 темно-серые, Арт. BM...",84.29,NaN,Lenta
49350,"Носки мужские INWIN р. 29 черные, Арт. BMS02-01",84.29,NaN,Lenta
49351,"Носки женские INWIN р. 23–25 40 den, natural, ...",84.29,NaN,Lenta
49352,"Носки мужские INWIN средняя посадка р. 27–29, ...",73.68,NaN,Lenta


In [3]:
products = products.drop_duplicates() # remove duplicates
products

,name,price,pricing_unit,supermarket
0,"Макароны PASTA ZARA Спагетти, 500г",121.05,500г,Lenta
1,"Макароны PASTA ZARA Рожки №27, 500г",121.05,500г,Lenta
2,Макароны ШЕБЕКИНСКИЕ Вермишель легкая группа А...,73.69,450г,Lenta
3,"Макароны ШЕБЕКИНСКИЕ Спирали группа А, высший ...",75.49,450г,Lenta
4,Лапша яичная РОЛЛТОН Классическая группа В выс...,78.94,400г,Lenta
...,...,...,...,...
49338,"Носки женские INWIN р. 35–37, белые, Арт. BWS0...",95.99,NaN,Lenta
49339,"Носки женские INWIN р. 38–40, белые, Арт. BWS0...",95.99,NaN,Lenta
49342,"Носки женские INWIN р. 38–40, цвет серый мелан...",95.99,NaN,Lenta
49343,"Носки женские INWIN р. 35–37, черные, Арт. BWS...",95.99,NaN,Lenta


In [6]:
product_regex_map = {
    'rice': r'^рис\b(?!.*овощ)',
    'bread': r'(^хлеб\b|^багет\b|^батон\b)(?!.*печеноч)',
    'chicken_fillet': r'^филе (кур|груд)(?!.*(копч|соус|запеч|бедр|индей|утен))',
    'pork_leg': r'^окорок (свин|из свин)(?!.*(копч|соус))',
    'egg': r'^яйцо курин',
    'cucumber': r'^огур(цы|ец)(?!.*(солен|маринован|ягод))',
    'carrot': r'^морковь(?!.*(корей|отвар))',
    'onion': r'^лук репч(?!.*суш)',
    'tomato': r'^томаты(?!.*(сок|очищ|маринован|вялен|солен|измельч|кус))',
    'cabbage': r'^капуста\b.*белокоч',
    'eggplant': r'^баклажаны,',
    'banana': r'^банан(?!.*(вял|суш|куб))',
    'orange': r'^апельсин(?!.*(сахар|куб))',
    'milk': r'^молоко(?!.*(сгущ|кокос|сух|топл|коз|обогащ|витам|лактоз|печен))',
    'yogurt': r'^йогурт\b(?!.*питье)',
    'condensed_milk': r'^молоко.*сгущ(?!.*(варен|кофе|какао))',
    'green_tea': r'^чай зел(?!.*порош)',
    'black_tea': r'^чай черн',
    'ground_coffee': r'^кофе(?!.*(капсул|раствор|фильтр)).*молот',
    'sugar': r'^сахар\b(?!.*(ванил|коричн))',
    'salt': r'^соль(?!.*(посуд|ванн|розов|чесн|прян|купан|мельн))',
    'sunflower_oil': r'^масло\b(?!.*(оливк|спрей|аром|вкус|добавл)).*подсолн',
    'water': r'^вода\b(?!.*(малин|клюкв|лимон|цитр)).*негаз',
    'buckwheat': r'(^крупа\b.*гречн|^гречка\b)(?!.*зел)',
    'spaghetti': r'^макароны\b.*спагетти',
    'rice_noodles': r'(^лапша|^вермишель).*(рис|фунчоз)(?!.*соус)',
    'tofu': r'(^продукт)(?!.*(копч|папр)).*тофу|^тофу(?!.*(гриб|томат))',
    'mango': r'^манго желт',
    'fish_sauce': r'^соус.*рыбн'
}
product_regex_list = '|'.join(product_regex_map.values())
product_regex_list

'^рис\\b(?!.*овощ)|(^хлеб\\b|^багет\\b|^батон\\b)(?!.*печеноч)|^филе (кур|груд)(?!.*(копч|соус|запеч|бедр|индей|утен))|^окорок (свин|из свин)(?!.*(копч|соус))|^яйцо курин|^огур(цы|ец)(?!.*(солен|маринован|ягод))|^морковь(?!.*(корей|отвар))|^лук репч(?!.*суш)|^томаты(?!.*(сок|очищ|маринован|вялен|солен|измельч|кус))|^капуста\\b.*белокоч|^баклажаны,|^банан(?!.*(вял|суш|куб))|^апельсин(?!.*(сахар|куб))|^молоко(?!.*(сгущ|кокос|сух|топл|коз|обогащ|витам|лактоз|печен))|^йогурт\\b(?!.*питье)|^молоко.*сгущ(?!.*(варен|кофе|какао))|^чай зел(?!.*порош)|^чай черн|^кофе(?!.*(капсул|раствор|фильтр)).*молот|^сахар\\b(?!.*(ванил|коричн))|^соль(?!.*(посуд|ванн|розов|чесн|прян|купан|мельн))|^масло\\b(?!.*(оливк|спрей|аром|вкус|добавл)).*подсолн|^вода\\b(?!.*(малин|клюкв|лимон|цитр)).*негаз|(^крупа\\b.*гречн|^гречка\\b)(?!.*зел)|^макароны\\b.*спагетти|(^лапша|^вермишель).*(рис|фунчоз)(?!.*соус)|(^продукт)(?!.*(копч|папр)).*тофу|^тофу(?!.*(гриб|томат))|^манго желт|^соус.*рыбн'

In [7]:
filtered_products = products.loc[products.name.str.contains(product_regex_list, case=False, regex=True)]
filtered_products

C:\Users\Just Alex\AppData\Local\Temp\ipykernel_19792\4054580008.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtered_products = products.loc[products.name.str.contains(product_regex_list, case=False, regex=True)]


,name,price,pricing_unit,supermarket
0,"Макароны PASTA ZARA Спагетти, 500г",121.05,500г,Lenta
21,"Макароны DE CECCO Спагетти №12, 500г",382.49,500г,Lenta
35,"Макароны 365 ДНЕЙ Спагетти, группа В высший со...",23.49,400г,Lenta
37,"Макароны AIDA Спагетти группа А высший сорт, 500г",84.20,500г,Lenta
45,"Макароны AMERIA Спагетти №3, группа В высший с...",100.99,900г,Lenta
...,...,...,...,...
49311,"Яйцо куриное ЛЕНТА FRESH XXL СВ, 10шт",146.49,10шт,Lenta
49312,"Яйцо куриное ЛЕНТА FRESH С2, 10шт",97.99,10шт,Lenta
49313,"Яйцо куриное ПРАКСИС С1, 10шт",141.09,10шт,Lenta
49316,"Яйцо куриное СЕЛО ЗЕЛЁНОЕ Деревенское СО, 15шт",243.99,15шт,Lenta


In [8]:
def assign_product_type(row):
    name = row['name']
    for product_type, regex in product_regex_map.items():
        match = re.search(regex, name, flags=re.IGNORECASE)
        if match:
            return product_type
    return None

filtered_products = filtered_products.copy()  # recreate the dataframe
filtered_products.loc[:,'product_type'] = filtered_products.apply(assign_product_type, axis=1)
with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
    display(filtered_products)

,name,price,pricing_unit,supermarket,product_type
0,"Макароны PASTA ZARA Спагетти, 500г",121.05,500г,Lenta,spaghetti
21,"Макароны DE CECCO Спагетти №12, 500г",382.49,500г,Lenta,spaghetti
35,"Макароны 365 ДНЕЙ Спагетти, группа В высший сорт, 400г",23.49,400г,Lenta,spaghetti
37,"Макароны AIDA Спагетти группа А высший сорт, 500г",84.20,500г,Lenta,spaghetti
45,"Макароны AMERIA Спагетти №3, группа В высший сорт, 900г",100.99,900г,Lenta,spaghetti
46,"Макароны GRANMULINO Premium Спагетти, 400г",107.49,400г,Lenta,spaghetti
47,"Макароны безглютеновые FEDERICI Спагетти, 400г",210.59,400г,Lenta,spaghetti
74,"Макароны ШЕБЕКИНСКИЕ Спагеттини №001, 450г",79.99,450г,Lenta,spaghetti
88,"Макароны GRAND DI PASTA Спагетти, 450г",104.20,450г,Lenta,spaghetti
90,"Макароны MAKFA Спагетти, вермишель длинная, 450г",74.99,450г,Lenta,spaghetti


In [9]:
def extract_weight(row):
    name, pricing_unit = row['name'], row['pricing_unit']
    pricing_unit = str(pricing_unit) if pd.notna(pricing_unit) else "" # handle NaN values as empty
    
    # first, check name
    # calculate weight if there're multiple portions
    match = re.search(r'(\d+)(x|х)(\d+|\d+[,.]\d+)\s?г\b', name)
    if match:
        portion = float(match.group(1).replace(',', '.'))
        per_portion = float(match.group(3).replace(',', '.'))
        return portion * per_portion
    # if there's a single weight
    match = re.search(r'(\d+|\d+[,.]\d+)\s?(г\b|кг)', name)
    if match:
        weight = float(match.group(1).replace(',', '.'))
        unit = match.group(2)
        return weight * 1000 if unit == 'кг' else weight
    # if name doesn't contain anything, check pricing_unit
    match = re.search(r'(\d+|\d+[,.]\d+)\s?(г\b|кг)', pricing_unit)
    if match:
        weight = float(match.group(1).replace(',', '.'))
        unit = match.group(2)
        return weight * 1000 if unit == 'кг' else weight
    # if name contains a word "весовой", it means it's a price for 1 kilo
    match = re.search(r'\bвесов', name)
    if match:
        weight = 1000
        return weight

    return None  # if nothing matched

def extract_number_of_units(row):
    name, pricing_unit = row['name'], row['pricing_unit']
    pricing_unit = str(pricing_unit) if pd.notna(pricing_unit) else "" # handle NaN values as empty

    # first, check name
    match = re.search(r'(\d+)\s?(шт|пак|пир|саш)', name)
    if match:
        number_of_units = int(match.group(1))
        return number_of_units
    # if name doesn't contain anything, check pricing_unit
    match = re.search(r'(\d+)\s?(шт|пак|пир|саш)', pricing_unit)
    if match:
        number_of_units = int(match.group(1))
        return number_of_units

    return None  # if nothing matched

def extract_volume(row):
    name, pricing_unit = row['name'], row['pricing_unit']
    pricing_unit = str(pricing_unit) if pd.notna(pricing_unit) else "" # handle NaN values as empty

    # first, check name
    # calculate volume if there're multiple portions
    match = re.search(r'(\d+)(x|х)(\d+|\d+[,.]\d+)\s?мл', name)
    if match:
        portion = float(match.group(1).replace(',', '.'))
        per_portion = float(match.group(3).replace(',', '.'))
        return portion * per_portion
    # if there's a single volume
    match = re.search(r'(\d+|\d+[,.]\d+)\s?(мл|л\b)', name)
    if match:
        volume = float(match.group(1).replace(',', '.'))
        unit = match.group(2)
        return volume * 1000 if unit == 'л' else volume
    # if name doesn't contain anything, check pricing_unit
    match = re.search(r'(\d+|\d+[,.]\d+)\s?(мл|л\b)', pricing_unit)
    if match:
        volume = float(match.group(1).replace(',', '.'))
        unit = match.group(2)
        return volume * 1000 if unit == 'л' else volume

    return None  # if nothing matched

filtered_products = filtered_products.copy()  # recreate the dataframe
# extract and calculate weights
filtered_products.loc[:,'weight'] = filtered_products.apply(extract_weight, axis=1)  # a column with weigths in grams
filtered_products.loc[:,'price_kg'] = filtered_products.price / filtered_products.weight * 1000   # a column with prices per kg
# extract and calculate number of units (for products with "шт")
filtered_products.loc[:,'number_of_units'] = filtered_products.apply(extract_number_of_units, axis=1)  # a column with number of units
filtered_products.loc[:,'price_unit'] = filtered_products.price / filtered_products.number_of_units   # a column with prices per unit
# extract and calculate volume
filtered_products.loc[:,'volume'] = filtered_products.apply(extract_volume, axis=1)  # a column with volume in ml
filtered_products.loc[:,'price_lit'] = filtered_products.price / filtered_products.volume * 1000   # a column with prices per liter
with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
    display(filtered_products)

,name,price,pricing_unit,supermarket,product_type,weight,price_kg,number_of_units,price_unit,volume,price_lit
0,"Макароны PASTA ZARA Спагетти, 500г",121.05,500г,Lenta,spaghetti,500.0,242.100000,NaN,NaN,NaN,NaN
21,"Макароны DE CECCO Спагетти №12, 500г",382.49,500г,Lenta,spaghetti,500.0,764.980000,NaN,NaN,NaN,NaN
35,"Макароны 365 ДНЕЙ Спагетти, группа В высший сорт, 400г",23.49,400г,Lenta,spaghetti,400.0,58.725000,NaN,NaN,NaN,NaN
37,"Макароны AIDA Спагетти группа А высший сорт, 500г",84.20,500г,Lenta,spaghetti,500.0,168.400000,NaN,NaN,NaN,NaN
45,"Макароны AMERIA Спагетти №3, группа В высший сорт, 900г",100.99,900г,Lenta,spaghetti,900.0,112.211111,NaN,NaN,NaN,NaN
46,"Макароны GRANMULINO Premium Спагетти, 400г",107.49,400г,Lenta,spaghetti,400.0,268.725000,NaN,NaN,NaN,NaN
47,"Макароны безглютеновые FEDERICI Спагетти, 400г",210.59,400г,Lenta,spaghetti,400.0,526.475000,NaN,NaN,NaN,NaN
74,"Макароны ШЕБЕКИНСКИЕ Спагеттини №001, 450г",79.99,450г,Lenta,spaghetti,450.0,177.755556,NaN,NaN,NaN,NaN
88,"Макароны GRAND DI PASTA Спагетти, 450г",104.20,450г,Lenta,spaghetti,450.0,231.555556,NaN,NaN,NaN,NaN
90,"Макароны MAKFA Спагетти, вермишель длинная, 450г",74.99,450г,Lenta,spaghetti,450.0,166.644444,NaN,NaN,NaN,NaN


In [248]:
filtered_products.to_csv(f'filtered_products-2025-03-08-lenta.csv')